In [ ]:
!pip install soundfile
import os
import numpy as np
import pandas as pd
import scipy.signal as signal
import soundfile as sf
import matplotlib.pyplot as plt
import IPython.display as ipd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
MFCC_array = np.load('/content/drive/My Drive/TDL_DATASET/MFCC_data_large.npy').astype(np.float32)

In [ ]:
language_dummies = np.load('/content/drive/My Drive/TDL_DATASET/language_dummy_large.npy').astype(np.float32)

In [ ]:
X_train_tf,X_test_tf,y_train_tf,y_test_tf = train_test_split(
    MFCC_array,language_dummies,stratify = language_dummies,test_size = 0.10,random_state = 0)

In [ ]:
X_train_tf = X_train_tf.reshape(-1,1000,40,1)

In [ ]:
X_test_tf = X_test_tf.reshape(-1,1000,40,1)

In [ ]:
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix

from keras.models import Model, load_model, Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten
from keras.layers import Dropout, Input, Activation
from keras.optimizers import Nadam, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.models import load_model
from keras.layers.normalization import BatchNormalization
from keras import regularizers

Using TensorFlow backend.


In [ ]:
input_shape = (1000,40,1)
model = Sequential()
model.add(Conv2D(
        32,
        (3, 3),
        strides=(1, 1),
        padding='same',
        #kernel_regularizer=regularizers.l2(0.001),
        input_shape=input_shape,data_format = 'channels_last'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
model.add(Conv2D(
        64,
        (3, 3),
        strides=(1, 1),
        padding='same'))
        #kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
model.add(AveragePooling2D(
        pool_size=(3, 3),
        strides=(2, 2),
        padding='same'))
model.add(Flatten())

model.add(Dense(
        128,
        activation='relu'))
       # kernel_regularizer=regularizers.l2(0.001)))

model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(3))
model.add(Activation('softmax'))


model.compile(
        loss='categorical_crossentropy',
        optimizer='Adadelta',
        metrics=['accuracy'])














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [ ]:
checkpoint = ModelCheckpoint(
                'model.h5',
                monitor='val_loss',
                verbose=0,
                save_best_only=True,
                mode='min')
#es = EarlyStopping(monitor='val_loss',mode = 'max')
model.fit(
                X_train_tf,
                y_train_tf,
                epochs=100,
                callbacks=[checkpoint],
                verbose=1,
                validation_data=(X_test_tf, y_test_tf),
                batch_size=32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3288 samples, validate on 366 samples
Epoch 1/100
3288/3288 [==============================] - 8s 2ms/step - loss: 1.2317 - acc: 0.4097 - val_loss: 1.0661 - val_acc: 0.4563
Epoch 2/100
3288/3288 [==============================] - 4s 1ms/step - loss: 0.9587 - acc: 0.5484 - val_loss: 1.1350 - val_acc: 0.4180
Epoch 3/100
3288/3288 [==============================] - 4s 1ms/step - loss: 0.8072 - acc: 0.6481 - val_loss: 0.9488 - val_acc: 0.5273
Epoch 4/100
3288/3288 [==============================] - 4s 1ms/step - loss: 0.6587 - acc: 0.7281 - val_loss: 1.0443 - val_acc: 0.4590
Epoch 5/100
3288/3288 [==============================] - 4s 1ms/step - loss: 0.5509 - acc: 0.7874 - val_loss: 0.7975 - val_acc: 0.6721
Epoch 6/100
3288/3288 [==============================] - 4s 1ms/step - loss: 0.4048 - acc: 0.8495 - val_loss: 0.9470 - val_acc: 0.6120
Epoch 7/100
3288/3288 [========================

In [ ]:
model.save('/content/drive/My Drive/TDL_DATASET/Model_CNN.h5')

In [ ]:
y_pred = model.predict(X_test_tf)
y_test1 = []
for i in range(0,len(y_test_tf)):
    argmax = np.argmax(y_test_tf[i,:])
    y_test1.append(argmax)

In [ ]:
y_pred1 = []
for i in range(0,len(y_test_tf)):
    argmax = np.argmax(y_pred[i,:])
    y_pred1.append(argmax)

In [ ]:
confusion_matrix(y_test1,y_pred1)

array([[110,   3,   9],
       [ 37,  80,   5],
       [ 40,   5,  77]])

In [ ]:
print(classification_report(y_test1,y_pred1))

              precision    recall  f1-score   support

           0       0.59      0.90      0.71       122
           1       0.91      0.66      0.76       122
           2       0.85      0.63      0.72       122

    accuracy                           0.73       366
   macro avg       0.78      0.73      0.73       366
weighted avg       0.78      0.73      0.73       366



In [ ]:
MFCC_array_t = np.load('/content/drive/My Drive/TDL_DATASET/MFCC_data_t_large.npy').astype(np.float32)

In [ ]:
MFCC_array_t = MFCC_array_t.reshape(-1,1000,40,1)

In [ ]:
predictions = model.predict(MFCC_array_t)

In [ ]:
predictions

array([[9.7085518e-01, 2.6281331e-02, 2.8634861e-03],
       [9.5298678e-01, 4.7007684e-02, 5.5580990e-06],
       [9.9807811e-01, 1.3293664e-03, 5.9253076e-04],
       ...,
       [1.8864377e-04, 1.5700872e-09, 9.9981135e-01],
       [1.9976765e-06, 2.6910028e-11, 9.9999797e-01],
       [9.2108892e-03, 1.2477577e-06, 9.9078780e-01]], dtype=float32)

In [ ]:
y_pred1 = []
for i in range(0,len(predictions)):
    argmax = np.argmax(predictions[i,:])
    y_pred1.append(argmax)

In [ ]:
y_test1 = []
language_dummies_t=np.load('/content/drive/My Drive/TDL_DATASET/language_dummy_t_large.npy').astype(np.float32)
for i in range(0,len(language_dummies_t)):
    argmax = np.argmax(language_dummies_t[i,:])
    y_test1.append(argmax)

In [ ]:
confusion_matrix(y_test1,y_pred1)

array([[155,  21,   4],
       [ 72,  67,  41],
       [ 30,  12, 138]])

In [ ]:


print(classification_report(y_test1,y_pred1))



              precision    recall  f1-score   support

           0       0.60      0.86      0.71       180
           1       0.67      0.37      0.48       180
           2       0.75      0.77      0.76       180

    accuracy                           0.67       540
   macro avg       0.68      0.67      0.65       540
weighted avg       0.68      0.67      0.65       540



In [ ]:
def generate_fb_and_mfcc(signal, sample_rate):
    # Pre-Emphasis
    pre_emphasis = 0.97
    emphasized_signal = np.append(signal[0],signal[1:] - pre_emphasis * signal[:-1])
    # Framing
    frame_size = 0.025
    frame_stride = 0.01
    # Convert from seconds to samples
    frame_length, frame_step = (
        frame_size * sample_rate,
        frame_stride * sample_rate)
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    # Make sure that we have at least 1 frame
    num_frames = int(
        np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))

    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))

    # Pad Signal to make sure that all frames have equal
    # number of samples without truncating any samples
    # from the original signal
    pad_signal = np.append(emphasized_signal, z)

    indices = (
        np.tile(np.arange(0, frame_length), (num_frames, 1)) +
        np.tile(
            np.arange(0, num_frames * frame_step, frame_step),
            (frame_length, 1)
        ).T
    )
    frames = pad_signal[indices.astype(np.int32, copy=False)]

    # Window
    frames *= np.hamming(frame_length)

    # Fourier-Transform and Power Spectrum
    NFFT = 512

    # Magnitude of the FFT
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))

    # Power Spectrum
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))

    # Filter Banks
    nfilt = 40

    low_freq_mel = 0

    # Convert Hz to Mel
    high_freq_mel = (2595 * np.log10(1 + (sample_rate / 2) / 700))

    # Equally spaced in Mel scale
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)

    # Convert Mel to Hz
    hz_points = (700 * (10**(mel_points / 2595) - 1))
    bin = np.floor((NFFT + 1) * hz_points / sample_rate)

    fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   
        f_m = int(bin[m])             
        f_m_plus = int(bin[m + 1])    

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banks = np.dot(pow_frames, fbank.T)

    # Numerical Stability
    filter_banks = np.where(
        filter_banks == 0,
        np.finfo(float).eps,
        filter_banks)

    filter_banks = 20 * np.log10(filter_banks)

    return filter_banks

In [ ]:
def predict(file_name,model):
  flac, samplerate = sf.read("/content/drive/My Drive/TDL_DATASET/"+file_name)
  series=flac
  length=samplerate
  MFCC = generate_fb_and_mfcc(series,length)
  sc = StandardScaler()
  MFCC_sc = sc.fit_transform(MFCC)
  data=MFCC_sc.reshape(-1,1000,40,1)
  return np.argmax(model.predict(data))

In [ ]:
model.load_weights("/content/drive/My Drive/TDL_DATASET/Model_CNN.h5")

In [ ]:
np.argmax(model.predict(X_test_tf)[0])

2

In [ ]:
language_dummies[0]

array([0., 1., 0.], dtype=float32)

In [ ]:
files=[ele for ele in os.listdir("/content/drive/My Drive/TDL_DATASET")  if ele.endswith(".flac")]

In [ ]:
files

['en_f_67a0cba10d171b24039a79faa1d4d603.fragment77.flac',
 'es_f_50298ab71aaba8508ebeef49d853df11.fragment86.flac',
 'en_m_b74b2bf2af570393cae91f4ed89cece7.fragment70.flac',
 'es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment15.flac',
 'de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment1.flac',
 'de_m_923551d571cc437382d0294dda2dd0aa.fragment21.flac']

In [ ]:
#english [0,1,0]
#spanish  [0,0,1]
#german   [1,0,0]
for ele in files:
  print(predict(ele,model))

0
1
1
2
0
0
